In [2]:
import requests
import pandas as pd
import bs4 
from bs4 import BeautifulSoup

In [27]:
df_loinc = pd.read_csv('dat/loinc.csv')
df_loinc.head()

,Entidad,LOINC
0,Laboratorio,44785-4
1,Laboratorio,1751-7
2,Laboratorio,88112-8
3,Laboratorio,42719-5
4,Laboratorio,2162-6


In [15]:
range(len(df_loinc))

range(0, 62)

In [5]:
# Empezamos poco a poco, haciendo una requests de uno de los códigos.

codigo = df_loinc['LOINC'][0]
codigo

'44785-4'

In [8]:
url = 'https://loinc.org/' + codigo
url # Esta es la URL que queremos consultar, la del código.

#Comprobamos que el resultado nos lleva a una página web correctamente.

'https://loinc.org/44785-4'

In [9]:
page = requests.get('https://loinc.org/44785-4')
page # Comprobamos que podemos extraer los datos correctamente (respuesta 200).

<Response [200]>

In [10]:
soup = BeautifulSoup(page.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<script>var et_site_url='https://loinc.org';var et_post_id='101934';function et_core_page_resource_fallback(a,b){"undefined"===typeof b&&(b=a.sheet.cssRules&&0===a.sheet.cssRules.length);b&&(a.onerror=null,a.onload=null,a.href?a.href=et_site_url+"/?et_core_page_resource="+a.id+et_post_id:a.src&&(a.src=et_site_url+"/?et_core_page_resource="+a.id+et_post_id))}
</script>
<!-- This site is optimized with the Yoast SEO plugin v15.2.1 - https://yoast.com/wordpress/plugins/seo/ -->
<title>LOINC 44785-4 — Alanine aminotransferase [Enzymatic activity/volume] (Maximum value during study) in Serum or Plasma by No addition of P-5'-P</title>
<meta content="LOINC Code 44785-4 Alanine aminotransferase [Enzymatic activity/volume] (Maximum value during study) in Serum or Plasma by No addition of P-5'-P" name="description"/>
<meta content="index, follow, max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots"/>
<link href="https://loinc.or

In [13]:
# Queremos el nombre que le corresponde al código. Inspeccionamos el nombre en la web.
# Vemos que tiene un id, esto es ideal para filtrar el resultado del total, pero no podemos hacerlo como antes.
# Vemos que le corresponde el tab "scan" y tenemos que aprender a buscar índices (documentación)

nombre = soup.find(id='lcn') # Primero comprobamos que la función find nos da la información con el nombre y tal, que es lo que
                             # nos interesaba. Después ya podemos almacenarlo en una variable.
# *Nota. class en python es un nombre reservado, por lo que soup usa class_ como argumento y no class.
# find me da el elemento y find_all me devuelve un iterable (lo que al final tenemos que pasar a una lista, así que es 
# como si fuera una lista).

# Como lo que queremos es el contenido que nos interesa, lo filtramos con get_text:
nombre.get_text()


"Alanine aminotransferase [Enzymatic activity/volume] (Maximum value during study) in Serum or Plasma by No addition of P-5'-P"

In [24]:
codigos = df_loinc['LOINC'] # Almacenamos los códigos aparte.

ls_nombres =[] # Paso 0: creamos una lista para almacenar los nombres, que es nuestro objetivo final.

for cod in codigos:
    
    #Paso1: Generar url que tengo que leer.
    url = 'https://loinc.org/' + cod
    #Paso2: Leer esa url (verificar que se ha leído)
    page = requests.get(url)
    if page.status_code == 200:
        #Paso3: hacer el soup.
        soup = BeautifulSoup(page.content, 'html.parser') # html.parser se usa para indicarle que le hemos dado un html.
        #Paso 4: encontrar el nombre e inspeccionarlo para conseguir el texto.
        nombre = soup.find(id='lcn').get_text()
        
    else:
        nombre ='error' # Esto es importante por si hay errores, para que la lista final 
                        # tenga los nombres en la misma posición que estaban los códigos correspondientes.
    #Paso5: meterlo en una lista.
    ls_nombres.append(nombre)
ls_nombres[0:10]

# Tarda porque está leyendo un montón de páginas web, las abre y tal.
# Lo suyo para hacerlo poco a poco es ir comprobando cada paso con un print.

["Alanine aminotransferase [Enzymatic activity/volume] (Maximum value during study) in Serum or Plasma by No addition of P-5'-P",
 'Albumin [Mass/volume] in Serum or Plasma',
 "Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma by No addition of P-5'-P",
 'Bilirubin.total [Mass/volume] in Blood',
 'Creatinine [Mass/time] in 24 hour Urine',
 'Fibrin D-dimer [Units/volume] in Platelet poor plasma',
 'Ferritin [Moles/volume] in Serum or Plasma',
 'Fibrinogen [Mass/volume] in Platelet poor plasma by Coagulation assay',
 'Cholesterol in HDL [Mass/volume] in Serum or Plasma',
 'Hematocrit [Volume Fraction] of Blood']

In [28]:
df_loinc['Nombres'] = ls_nombres
df_loinc.head()

,Entidad,LOINC,Nombres
0,Laboratorio,44785-4,Alanine aminotransferase [Enzymatic activity/v...
1,Laboratorio,1751-7,Albumin [Mass/volume] in Serum or Plasma
2,Laboratorio,88112-8,Aspartate aminotransferase [Enzymatic activity...
3,Laboratorio,42719-5,Bilirubin.total [Mass/volume] in Blood
4,Laboratorio,2162-6,Creatinine [Mass/time] in 24 hour Urine
